In [49]:
import os
import pandas as pd

def extract_folder_info(folder_name):
    """Extract TIME and INSTANCE from the folder name."""
    try:
        first_underscore = folder_name.index('_')
        last_underscore = folder_name.rindex('_')
        TIME = folder_name[:first_underscore]
        INSTANCE = folder_name[first_underscore + 1:last_underscore]
        return TIME, INSTANCE
    except ValueError:
        return None, None

def extract_gap(result_file_path):
    """Extract GAP value from results.txt file."""
    GAP = 0
    if os.path.exists(result_file_path):
        try:
            with open(result_file_path, 'r') as result_file:
                for line in result_file:
                    if line.startswith("GapClosed:"):
                        GAP = float(line.split(':')[1].strip())
                        break
        except Exception as e:
            print(f"Error reading {result_file_path}: {e}")
    return float(GAP)

def extract_feasibility(result_file_path):
    """Extract feasibility information from results.txt file."""
    feasibility = "false"  # Default value if line is not found
    if os.path.exists(result_file_path):
        try:
            with open(result_file_path, 'r') as result_file:
                for line in result_file:
                    if line.startswith("DbgSolFeasible:"):
                        feasibility = line.split(':')[1].strip().lower()
                        break
        except Exception as e:
            print(f"Error reading {result_file_path}: {e}")
    return feasibility

def extract_solving_time(solving_stat_file_path):
    """Extract SOLVING_TIME value from solving_statistic.txt file."""
    SOLVING_TIME = 3700
    if os.path.exists(solving_stat_file_path):
        try:
            with open(solving_stat_file_path, 'r') as solving_stat_file:
                for line in solving_stat_file:
                    if line.startswith("Total Time"):
                        SOLVING_TIME = float(line.split(':')[1].strip())
                        break
        except Exception as e:
            print(f"Error reading {solving_stat_file_path}: {e}")
    return SOLVING_TIME

def process_folder(folder_path, folder_name, mode):
    """Process each folder to extract required information."""
    TIME, INSTANCE = extract_folder_info(folder_name)
    if TIME is None or INSTANCE is None:
        return None

    result_file_path = os.path.join(folder_path, 'results.txt')
    solving_stat_file_path = os.path.join(folder_path, 'solving_statistic.txt')

    GAP = extract_gap(result_file_path)
    FEASIBILITY = extract_feasibility(result_file_path)
    SOLVING_TIME = extract_solving_time(solving_stat_file_path)

    return [INSTANCE, mode, GAP, SOLVING_TIME, FEASIBILITY]

def fetchresult(main_dir, modes):
    """Main function to iterate through directories and collect data."""
    data = []

    for mode in os.listdir(main_dir):
        if mode not in modes: continue
        mode_path = os.path.join(main_dir, mode)
        if os.path.isdir(mode_path):
            for folder_name in os.listdir(mode_path):
                folder_path = os.path.join(mode_path, folder_name)
                if os.path.isdir(folder_path):
                    folder_data = process_folder(folder_path, folder_name, mode)
                    if folder_data:
                        data.append(folder_data)

    df = pd.DataFrame(data, columns=['INSTANCE', 'MODE', 'GAP', 'TIME','FEASIBILITY'])
    return df



In [50]:
main_dir = os.path.join(os.getcwd(), 'Data')
df = fetchresult(main_dir,["gomory","vpc"])
df

,INSTANCE,MODE,GAP,TIME,FEASIBILITY
0,uct-subprob,gomory,1.480140e-02,1.13,true
1,binkar10_1,gomory,6.759320e-02,0.09,false
2,traininstance6,gomory,0.000000e+00,5.22,false
3,roll3000,gomory,1.213880e-01,0.56,false
4,neos-3381206-awhea,gomory,1.554310e-15,0.17,true
...,...,...,...,...,...
79,gen-ip002,vpc,9.064810e-03,0.17,true
80,cod105,vpc,4.375580e-03,165.90,true
81,nu25-pr12,vpc,4.432720e-02,222.17,true
82,icir97_tension,vpc,2.015170e-01,263.58,false


In [54]:
# Piddvot the DataFrame to the desired format
df_pivot = df.pivot(index='INSTANCE', columns='MODE', values=['GAP', 'TIME','FEASIBILITY'])

# Flatten the multi-level columns
df_pivot.columns = ['_'.join(col).strip() for col in df_pivot.columns.values]

gap_columns = [col for col in df_pivot.columns if 'GAP' in col]
print(gap_columns)
df_pivot[gap_columns] = df_pivot[gap_columns]*100
df_pivot[gap_columns] = df_pivot[gap_columns].round(2)

['GAP_gomory', 'GAP_vpc']


In [59]:
df_pivot

,GAP_gomory,GAP_vpc,TIME_gomory,TIME_vpc,FEASIBILITY_gomory,FEASIBILITY_vpc
INSTANCE,,,,,,
30n20b8,9.68264,0.0,29.19,3669.05,false,false
beasleyC3,1.17961,1.17961,0.08,131.38,true,true
binkar10_1,6.75932,6.69134,0.09,41.79,false,false
bnatt400,0.0,0.0,1.94,3558.22,true,true
cod105,0.865617,0.437558,4.26,165.9,true,true
cost266-UUE,27.2712,18.6341,0.37,300.45,true,true
csched007,8.92488,3.38965,0.27,237.61,false,false
exp-1-500-5-5,23.1601,21.7312,0.06,36.89,true,true
gen-ip002,2.14825,0.906481,0.01,0.17,true,true


In [53]:
df_rounded

NameError: name 'df_rounded' is not defined